<h2>Conexão</h2>

In [ ]:
class Conexao:
#classe conexão com funções para conectar ao banco SQL  

     #Informações para a conexão
    jdbcHostname = '<NOMEDOHOST>.database.windows.net'
    jdbcDatabase = '<NOME DO DB>'
    jdbcPort = 1433 #numero da porta
    jdbcUsername = '<USERNAME>'
    jdbcPassword = dbutils.secrets.get(scope = '<nome do escopo do KV>', key = '<nome da chave>')
    jdbcUrl = f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}'
    
    def conectar(jdbcUsername=jdbcUsername, jdbcPassword = jdbcPassword ):
        #retorna informações de conexão com o SQL
        connectionProperties = {
          "user" : jdbcUsername,
          "password" : jdbcPassword,
          "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
            }
        return connectionProperties
        
    def salvar(var, nome_tabela, schema, jdbcUrl=jdbcUrl,jdbcUsername = jdbcUsername, jdbcPassword = jdbcPassword):
        #salva um dataframe (var) com o nome schema.nome_tabela no banco de dados
        var.write.format('jdbc')\
                        .mode('overwrite')\
                        .option('url', jdbcUrl)\
                        .option('dbtable',  f'{schema}.{nome_tabela}' )\
                        .option("user", jdbcUsername) \
                        .option("password", jdbcPassword) \
                        .save()  
        
    def url_sql (jdbcUrl = jdbcUrl):
        #retorna somente a URL do banco, caso precise
        return jdbcUrl
    
    def retorna_df (nome_tabela_sql):
        #falta testar, deve funcionar
        conexao = Conexao.conectar()
        df = spark.read.jdbc(url=jdbcUrl, table=nome_tabela_sql, properties=conexao)
        return df

<h4>Mount and unmount (Azure/S3)</h4>

In [ ]:
#Azure
dbutils.fs.mount(
   source = f"wasbs://{nome_container}@{storage}.blob.core.windows.net",
   mount_point=f"/mnt/{nome_container}",
   extra_configs={f"fs.azure.account.key.{storage}.blob.core.windows.net":key})

dbutils.fs.unmount(f"/mnt/{nome_container}")


#S3 
aws_bucket_name = "<aws-bucket-name>"
mount_name = "<mount-name>"
dbutils.fs.mount(f"s3a://{aws_bucket_name}", f"/mnt/{mount_name}")
display(dbutils.fs.ls(f"/mnt/{mount_name}"))

<h4> Uso da Biblioteca MSAL, pra Banco de dados <h4>


In [ ]:
import msal

class AzureSQLWriter:
    def __init__(self, tenant_id, service_principal_id, service_principal_secret, jdbc_hostname, jdbc_database, jdbc_port):
        self.tenant_id = tenant_id
        self.service_principal_id = service_principal_id
        self.service_principal_secret = service_principal_secret
        self.jdbc_hostname = jdbc_hostname
        self.jdbc_database = jdbc_database
        self.jdbc_port = jdbc_port
        self.msal_authority = f'https://login.microsoftonline.com/{self.tenant_id}'
        self.resource_url = 'https://database.windows.net/.default'

    def get_access_token(self):
        app = msal.ConfidentialClientApplication(
            self.service_principal_id,
            authority=self.msal_authority,
            client_credential=self.service_principal_secret
        )
        result = app.acquire_token_for_client(scopes=[self.resource_url])
        access_token = result.get('access_token')
        return access_token

    def write_to_azure_sql(self, dataframe, table_name):
        access_token = self.get_access_token()
        jdbc_url = f'jdbc:sqlserver://{self.jdbc_hostname}:{self.jdbc_port};database={self.jdbc_database}'

        dataframe.write \
            .format('jdbc') \
            .mode('overwrite') \
            .option('url', jdbc_url) \
            .option('dbtable', table_name) \
            .option('accessToken', access_token) \
            .save()


# Parâmetros de exemplo
tenant_id = ''
service_principal_id = dbutils.secrets.get(scope='', key='')
service_principal_secret = dbutils.secrets.get(scope='', key='')
jdbc_hostname = ''
jdbc_database = ''
jdbc_port = 
table_name = ''

# Criando uma instância de AzureSQLWriter
sql_writer = AzureSQLWriter(tenant_id, service_principal_id, service_principal_secret, jdbc_hostname, jdbc_database, jdbc_port)

# Escrevendo o dataframe no Azure SQL Database
sql_writer.write_to_azure_sql(df, table_name)